# Nova Model Whiteboard

This code is just me experimenting with building my own neural network, none of this has been unit tested, time or memory efficiency optimized, etc...

In [1]:
import numpy as np
import tensorflow as tf
from neuralnetworks import ffnn, embedding, attention, masking, transformer, constants
import importlib

2024-07-27 21:28:01.542055: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Embedding Model Tests

In [2]:
importlib.reload(embedding)

<module 'neuralnetworks.embedding' from '/Users/joericks/Desktop/nova/model/neuralnetworks/embedding.py'>

In [24]:
e_dim = constants.embd_constants.dimension

e_layer = embedding.EmbeddingLayer(e_dim, name = "embedding")

e_layer("hello")
e_layer("world")

<tf.Tensor: shape=(256,), dtype=float32, numpy=
array([ 1.5942723e+00, -6.7547476e-01, -2.6637869e+00, -2.3073845e-01,
        2.3068399e+00, -1.7264534e+00, -1.0379984e+00, -1.4703115e+00,
       -2.4398067e+00,  1.0663488e+00,  4.7101808e-01, -9.8482651e-01,
        1.9035375e+00,  2.2587671e+00, -6.6063637e-01,  7.0880644e-02,
        1.2899162e+00, -8.4084612e-01, -6.9391769e-01,  1.9873365e+00,
        2.5168214e+00,  2.6040128e-01, -6.5319583e-02, -8.7325698e-01,
        1.8292906e+00, -1.7624538e-01, -1.6859952e-01,  5.8000642e-01,
       -5.1502746e-02,  6.6792262e-01, -5.1596463e-02,  1.8066067e-03,
        2.3532717e+00,  2.4322982e+00,  4.9038255e-01, -4.5623708e-01,
       -9.1632295e-01,  1.0419813e+00, -4.4167089e-01, -1.7496036e-01,
       -1.4974694e+00, -4.6423760e-01, -9.8539364e-01, -5.4634824e-02,
       -1.5379286e+00,  7.0096117e-01, -2.4192855e+00, -2.0769010e+00,
       -8.8361102e-01, -8.3318537e-01, -2.1546333e-01, -2.2244570e+00,
        8.1321323e-01,  2.616

In [26]:
other_layer = embedding.EmbeddingLayer(e_dim, name = "other_embedding")

other_layer("!")

other_layer("=")

other_layer (";")

new_layer = e_layer + other_layer

In [27]:
new_layer.h

{'hello': 0, 'world': 1, '!': 2, '=': 3, ';': 4}

Documentation on softmax and ReLU are provided below: <br>
**[ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))** <br>
**[softmax](https://en.wikipedia.org/wiki/Softmax_function)**

In [28]:
#testing the performer attention head

sentence = ['hello', 'world']

idxs = [new_layer.h[s] for s in sentence if s in new_layer.h.keys()]

In [29]:
c = []

for i in idxs:
    c.append(new_layer.embeddings[i].numpy())

In [30]:
# then this process would get iterated based on the batches of training data passed through the model,
# but here we are going to cheat and just make this example be one batch

b = []

b.append(c)

batches = tf.constant(b)


In [31]:
batches

<tf.Tensor: shape=(1, 2, 256), dtype=float32, numpy=
array([[[ 1.5942723e+00, -6.7547476e-01, -2.6637869e+00, -2.3073845e-01,
          2.3068399e+00, -1.7264534e+00, -1.0379984e+00, -1.4703115e+00,
         -2.4398067e+00,  1.0663488e+00,  4.7101808e-01, -9.8482651e-01,
          1.9035375e+00,  2.2587671e+00, -6.6063637e-01,  7.0880644e-02,
          1.2899162e+00, -8.4084612e-01, -6.9391769e-01,  1.9873365e+00,
          2.5168214e+00,  2.6040128e-01, -6.5319583e-02, -8.7325698e-01,
          1.8292906e+00, -1.7624538e-01, -1.6859952e-01,  5.8000642e-01,
         -5.1502746e-02,  6.6792262e-01, -5.1596463e-02,  1.8066067e-03,
          2.3532717e+00,  2.4322982e+00,  4.9038255e-01, -4.5623708e-01,
         -9.1632295e-01,  1.0419813e+00, -4.4167089e-01, -1.7496036e-01,
         -1.4974694e+00, -4.6423760e-01, -9.8539364e-01, -5.4634824e-02,
         -1.5379286e+00,  7.0096117e-01, -2.4192855e+00, -2.0769010e+00,
         -8.8361102e-01, -8.3318537e-01, -2.1546333e-01, -2.2244570e+00

## Attention Mechanism Testing

In [65]:
importlib.reload(attention)
importlib.reload(masking)

<module 'neuralnetworks.masking' from '/Users/joericks/Desktop/nova/model/neuralnetworks/masking.py'>

In [66]:
attention_mech = attention.PerformerAttention(e_dim, 8)

attention_mech(batches, batches, batches)

<tf.Tensor: shape=(1, 2, 256), dtype=float32, numpy=
array([[[ 1.90462697e+00,  1.09904373e+00,  2.34858274e-01,
          2.73171812e-02, -1.14288902e+00,  6.37277186e-01,
          3.03631157e-01, -2.50724733e-01,  1.01427591e+00,
          7.65199959e-01,  3.06311965e-01, -1.60317749e-01,
          8.72218668e-01, -2.59193301e+00,  1.38080049e+00,
          6.24896348e-01,  8.24412286e-01,  1.72429729e+00,
         -3.00930709e-01,  6.72433972e-01, -4.45027024e-01,
         -1.43559486e-01,  6.19352400e-01, -6.96493030e-01,
         -3.60891938e-01,  5.11123598e-01, -3.22770327e-01,
         -1.19217479e+00,  3.33005637e-01, -1.32216871e+00,
          5.26056468e-01,  5.11994898e-01,  5.05759120e-01,
         -2.47323632e-01, -1.00092697e+00, -1.18380345e-01,
          7.29414344e-01, -1.01365435e+00, -7.37427771e-01,
          5.47942340e-01,  9.26142395e-01, -1.68629527e+00,
         -1.39879775e+00, -1.50698566e+00, -1.43786967e+00,
         -2.17193991e-01,  1.84895456e+00,  9.7

In [54]:
batches

<tf.Tensor: shape=(1, 2, 256), dtype=float32, numpy=
array([[[ 1.5942723e+00, -6.7547476e-01, -2.6637869e+00, -2.3073845e-01,
          2.3068399e+00, -1.7264534e+00, -1.0379984e+00, -1.4703115e+00,
         -2.4398067e+00,  1.0663488e+00,  4.7101808e-01, -9.8482651e-01,
          1.9035375e+00,  2.2587671e+00, -6.6063637e-01,  7.0880644e-02,
          1.2899162e+00, -8.4084612e-01, -6.9391769e-01,  1.9873365e+00,
          2.5168214e+00,  2.6040128e-01, -6.5319583e-02, -8.7325698e-01,
          1.8292906e+00, -1.7624538e-01, -1.6859952e-01,  5.8000642e-01,
         -5.1502746e-02,  6.6792262e-01, -5.1596463e-02,  1.8066067e-03,
          2.3532717e+00,  2.4322982e+00,  4.9038255e-01, -4.5623708e-01,
         -9.1632295e-01,  1.0419813e+00, -4.4167089e-01, -1.7496036e-01,
         -1.4974694e+00, -4.6423760e-01, -9.8539364e-01, -5.4634824e-02,
         -1.5379286e+00,  7.0096117e-01, -2.4192855e+00, -2.0769010e+00,
         -8.8361102e-01, -8.3318537e-01, -2.1546333e-01, -2.2244570e+00

## Transformer Testing

In [67]:
importlib.reload(transformer)

<module 'neuralnetworks.transformer' from '/Users/joericks/Desktop/nova/model/neuralnetworks/transformer.py'>

In [68]:
tfrmr = transformer.TransformerLayer(batches.shape[2], batches.shape[0], name = "test_transformer")

In [69]:
tfrmr(batches)

<tf.Tensor: shape=(1, 2, 256), dtype=float32, numpy=
array([[[-2.47179687e-01,  7.94698834e-01,  1.49509937e-01,
         -2.61787474e-02,  4.91069585e-01,  4.43606079e-01,
         -3.50294709e-01,  2.43365586e-01,  8.14202875e-02,
         -8.76349866e-01,  9.73849535e-01,  9.89735603e-01,
         -5.66576898e-01,  2.09448025e-01,  3.52945626e-01,
          1.25310946e+00,  2.80075312e-01,  9.53009427e-01,
          1.16790211e+00, -2.68275023e-01,  7.34771609e-01,
         -2.96277255e-01, -2.93406010e-01,  2.57651448e-01,
          2.20986873e-01,  2.35399917e-01,  5.22957265e-01,
         -2.57114172e-02,  6.59005523e-01,  4.50074404e-01,
          2.92877555e-01, -4.78223592e-01, -1.04573095e+00,
          3.98494124e-01,  5.84482431e-01, -5.51074505e-01,
          1.13486558e-01,  2.93397218e-01,  2.90842742e-01,
         -9.17613208e-02,  1.12605095e+00,  2.76766419e-01,
          6.25379920e-01,  6.01705909e-01, -1.09569259e-01,
         -3.57296824e-01, -7.16196716e-01,  1.0